<a href="https://colab.research.google.com/github/dajebbar/Neural-Oblivious-Decision-Ensembles/blob/main/Song_Popularity_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-addons --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L
import tensorflow_addons as tfa
from tensorflow_probability import distributions, stats

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
import joblib

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

url = 'https://www.kaggle.com/c/song-popularity-prediction/data'
od.download(url)

Skipping, found downloaded files in "./song-popularity-prediction" (use force=True to force download)


In [ ]:
import os

data_dir = './song-popularity-prediction'

os.listdir(data_dir)

['train.csv', 'sample_submission.csv', 'test.csv']

In [ ]:
train_csv = data_dir + '/train.csv'
test_csv = data_dir + '/test.csv'
submission_csv = data_dir + '/sample_submission.csv'

In [ ]:
data = pd.read_csv(train_csv)
print(data.shape)
data.head()

(40000, 15)


,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
0,0,212990.0,0.642286,0.856520,0.707073,0.002001,10.0,NaN,-5.619088,0,0.082570,158.386236,4,0.734642,0
1,1,NaN,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1,0.127358,102.752988,3,0.711531,1
2,2,193213.0,NaN,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0,0.052282,178.685791,3,0.425536,0
3,3,249893.0,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0,0.035618,128.715630,3,0.453597,0
4,4,165969.0,0.493017,NaN,0.740982,0.002033,10.0,0.094891,-2.684095,0,0.050746,121.928157,4,0.741311,0


In [ ]:
test = pd.read_csv(test_csv)
X_test = test.drop(['id'], axis=1)

In [ ]:
X = data.drop(['id', 'song_popularity'], axis=1)
y = data['song_popularity']

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

## Model

In [ ]:
@tf.function
def sparsemoid(inputs):
    return tf.clip_by_value(0.5 * inputs + 0.5, 0., 1.)


def get_binary_lookup_table(depth):
    # output: binary tensor [depth, 2**depth, 2]
    indices = tf.keras.backend.arange(0, 2**depth, 1)
    offsets = 2 ** tf.keras.backend.arange(0, depth, 1)
    bin_codes = (tf.reshape(indices, (1, -1)) // tf.reshape(offsets, (-1, 1)) % 2)
    bin_codes = tf.stack([bin_codes, 1 - bin_codes], axis=-1)
    bin_codes = tf.cast(bin_codes, 'float32')
    binary_lut = tf.Variable(initial_value=bin_codes, trainable=False)
    return binary_lut


def get_feature_selection_logits(n_trees, depth, dim):
    initializer = tf.keras.initializers.random_uniform()
    init_shape = (dim, n_trees, depth)
    init_value = initializer(shape=init_shape, dtype='float32')
    return tf.Variable(init_value, trainable=True)


def get_output_response(n_trees, depth, units):
    initializer = tf.keras.initializers.random_uniform()
    init_shape = (n_trees, units, 2**depth)
    init_value = initializer(init_shape, dtype='float32')
    return tf.Variable(initial_value=init_value, trainable=True)


def get_feature_thresholds(n_trees, depth):
    initializer = tf.ones_initializer()
    init_shape = (n_trees, depth)
    init_value = initializer(shape=init_shape, dtype='float32')
    return tf.Variable(init_value, trainable=True)


def get_log_temperatures(n_trees, depth):
    initializer = tf.ones_initializer()
    init_shape = (n_trees, depth)
    init_value = initializer(shape=init_shape, dtype='float32')
    return tf.Variable(initial_value=init_value, trainable=True)


def init_feature_thresholds(features, beta, n_trees, depth):
    sampler = distributions.Beta(beta, beta)
    percentiles_q = sampler.sample([n_trees * depth])

    flattened_feature_values = tf.map_fn(tf.keras.backend.flatten, features)
    percentile = stats.percentile(flattened_feature_values, 100*percentiles_q)
    feature_thresholds = tf.reshape(percentile, (n_trees, depth))
    return feature_thresholds


def init_log_temperatures(features, feature_thresholds):
    input_threshold_diff = tf.math.abs(features - feature_thresholds)
    log_temperatures = stats.percentile(input_threshold_diff, 50, axis=0)
    return log_temperatures

In [ ]:
class ObliviousDecisionTree(L.Layer):
    def __init__(self,n_trees=3, depth=4, units=1,threshold_init_beta=1.):
        super().__init__()
        self.initialized = False
        self.n_trees = n_trees
        self.depth = depth
        self.units = units
        self.threshold_init_beta = threshold_init_beta

    def build(self, input_shape):
        dim = input_shape[-1]
        n_trees, depth, units = self.n_trees, self.depth, self.units
        self.feature_selection_logits = get_feature_selection_logits(n_trees, depth, dim)
        self.feature_thresholds = get_feature_thresholds(n_trees, depth)
        self.log_temperatures = get_log_temperatures(n_trees, depth)
        self.binary_lut = get_binary_lookup_table(depth)
        self.response = get_output_response(n_trees, depth, units)

    def _data_aware_initialization(self, inputs):
        beta, n_trees, depth = self.threshold_init_beta, self.n_trees, self.depth
        feature_values = self._get_feature_values(inputs)
        feature_thresholds = init_feature_thresholds(feature_values, beta, n_trees, depth)
        log_temperatures = init_log_temperatures(feature_values, feature_thresholds)
        self.feature_thresholds.assign(feature_thresholds)
        self.log_temperatures.assign(log_temperatures)

    def _get_feature_values(self, inputs, training=None):
        feature_selectors = tfa.activations.sparsemax(self.feature_selection_logits)
        feature_values = tf.einsum('bi,ind->bnd', inputs, feature_selectors)
        return feature_values

    def _get_feature_gates(self, feature_values):
        threshold_logits = (feature_values - self.feature_thresholds)
        threshold_logits = threshold_logits * tf.math.exp(-self.log_temperatures)
        threshold_logits = tf.stack([-threshold_logits, threshold_logits], axis=-1)
        feature_gates = sparsemoid(threshold_logits)
        return feature_gates

    def _get_aggregated_response(self, feature_gates):
        aggregated_gates = tf.einsum('bnds,dcs->bndc', feature_gates, self.binary_lut)
        aggregated_gates = tf.math.reduce_prod(aggregated_gates, axis=-2)
        aggregated_response = tf.einsum('bnc,nuc->bnu', aggregated_gates, self.response)
        return aggregated_response

    def call(self, inputs, training=None):
        if not self.initialized:
            self._data_aware_initialization(inputs)
            self.initialized = True

        feature_values = self._get_feature_values(inputs)
        feature_gates = self._get_feature_gates(feature_values)
        aggregated_response = self._get_aggregated_response(feature_gates)
        response_averaged_over_trees = tf.reduce_mean(aggregated_response, axis=1)
        return response_averaged_over_trees

In [ ]:
class NODE(keras.Model):
    def __init__(self, units=1, n_layers=1, link=tf.identity, n_trees=3, tree_depth=4, threshold_init_beta=1):
        super().__init__()
        self.units = units
        self.n_layers = n_layers
        self.n_trees = n_trees
        self.tree_depth = tree_depth
        self.units = units
        self.threshold_init_beta = threshold_init_beta

        self.bn = L.BatchNormalization()
        self.ensemble = [ObliviousDecisionTree(
            n_trees=n_trees, depth=tree_depth, units=units,threshold_init_beta=threshold_init_beta
        ) for _ in range(n_layers)]
        self.link = link

    def call(self, inputs, training=None):
        x = self.bn(inputs, training=training)
        for tree in self.ensemble:
            h = tree(x)
            x = tf.concat([x, h], axis=1)
        return self.link(h)

In [ ]:
get_cat_pipeline = lambda: Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('encoder', OneHotEncoder(sparse=False))
])

get_num_pipeline = lambda: Pipeline([
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler())
])

In [ ]:
class model_config:
    NUMERIC_FEATURE_NAMES=[
        'song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
        'speechiness', 'tempo', 'audio_valence'
    ]
    CATEGORICAL_FEATURE_NAMES=[
        'key','audio_mode','time_signature'   
    ]

MAX_EPOCHS  = 100

get_callbacks = lambda : [
    keras.callbacks.EarlyStopping(min_delta=1e-4, patience=10, verbose=1, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(patience=3, verbose=1)
]

## Training

In [ ]:
preds = []

for fold, (train_index, valid_index) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    num_pipeline = get_num_pipeline().fit(X_train[model_config.NUMERIC_FEATURE_NAMES])
    cat_pipeline = get_cat_pipeline().fit(X_train[model_config.CATEGORICAL_FEATURE_NAMES])
    
    X_train = np.hstack((
        num_pipeline.transform(X_train[model_config.NUMERIC_FEATURE_NAMES]),
        cat_pipeline.transform(X_train[model_config.CATEGORICAL_FEATURE_NAMES])
    ))
    X_valid = np.hstack((
        num_pipeline.transform(X_valid[model_config.NUMERIC_FEATURE_NAMES]),
        cat_pipeline.transform(X_valid[model_config.CATEGORICAL_FEATURE_NAMES])
    ))
    X_test_ = np.hstack((
        num_pipeline.transform(X_test[model_config.NUMERIC_FEATURE_NAMES]),
        cat_pipeline.transform(X_test[model_config.CATEGORICAL_FEATURE_NAMES])
    ))
    
    model = NODE()
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    model.fit(
        X_train, y_train, validation_data=(X_valid, y_valid), callbacks=get_callbacks(), 
        epochs=MAX_EPOCHS
    )  
    preds.append(model.predict(X_test_))

Epoch 1/100
1125/1125 [==============================] - 4s 2ms/step - loss: 0.7901 - accuracy: 0.6356 - val_loss: 0.6611 - val_accuracy: 0.6357 - lr: 0.0010
Epoch 2/100
1125/1125 [==============================] - 2s 2ms/step - loss: 0.6564 - accuracy: 0.6356 - val_loss: 0.6594 - val_accuracy: 0.6357 - lr: 0.0010
Epoch 3/100
1125/1125 [==============================] - 2s 2ms/step - loss: 0.6561 - accuracy: 0.6356 - val_loss: 0.6587 - val_accuracy: 0.6357 - lr: 0.0010
Epoch 4/100
1125/1125 [==============================] - 2s 2ms/step - loss: 0.6560 - accuracy: 0.6356 - val_loss: 0.6570 - val_accuracy: 0.6357 - lr: 0.0010
Epoch 5/100
1125/1125 [==============================] - 2s 2ms/step - loss: 0.6560 - accuracy: 0.6356 - val_loss: 0.6570 - val_accuracy: 0.6357 - lr: 0.0010
Epoch 6/100
1125/1125 [==============================] - 2s 2ms/step - loss: 0.6559 - accuracy: 0.6356 - val_loss: 0.6589 - val_accuracy: 0.6357 - lr: 0.0010
Epoch 7/100
1125/1125 [=============================

## Submission

In [ ]:
submissions = pd.read_csv(submission_csv)
submissions['song_popularity'] = np.array(preds).mean(axis=0)
submissions.to_csv('preds.csv', index=False)